# Determine the best features using a Logistic Regression Model

## Imports

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    precision_score,
    accuracy_score,
)
from sklearn.model_selection import train_test_split
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns
import itertools

from joblib import dump, load

## Find the git root Directory

In [ ]:
# Get the current working directory
cwd = Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd

else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break

# Check if a Git root directory was found
if root_dir is None:
    raise FileNotFoundError("No Git root directory found.")

## Import Utilities

In [ ]:
sys.path.append(f"{root_dir}/utils")
import analysis_utils as au
import preprocess_utils as ppu
import eval_utils as eu
import train_utils as tu

# Seed and Generator for Reproducibility

In [ ]:
rnd_val = 0  # Random value for all seeds
rng = np.random.default_rng(seed=rnd_val)  # random number generator
plt.switch_backend("Agg")  # Switch to non-interactive backend

## Create the directory path if non-existent

In [ ]:
fig_out_path = Path(f"{root_dir}/2.evaluate_models/model_performance_figures")

if not fig_out_path.exists():
    fig_out_path.mkdir()

## Load Model

In [ ]:
models_path = Path(f"{root_dir}/2.evaluate_models/model_data/lr_plate3_cp_fs_data")

lr = load(models_path / "lr_model.joblib")

## Load Data

In [ ]:
testdf = load(models_path / "testdf.joblib")
le = load(models_path / "label_encoder.joblib")

## Make Predictions

In [1]:
testdf["preds"] = lr.predict(testdf.drop("label", axis="columns"))

---------------------------------------------------------------------
ValueError                          Traceback (most recent call last)
Cell In[16], line 1
----> 1 testdf["preds"] = lr.predict(testdf.drop("label", axis="columns"))

File ~/anaconda3/envs/nf1_analysis/lib/python3.9/site-packages/sklearn/linear_model/_base.py:309, in LinearClassifierMixin.predict(self, X)
    295 def predict(self, X):
    296     """
    297     Predict class labels for samples in X.
    298 
   (...)
    307         Predicted class label per sample.
    308     """
--> 309     scores = self.decision_function(X)
    310     if len(scores.shape) == 1:
    311         indices = (scores > 0).astype(int)

File ~/anaconda3/envs/nf1_analysis/lib/python3.9/site-packages/sklearn/linear_model/_base.py:288, in LinearClassifierMixin.decision_function(self, X)
    286 n_features = self.coef_.shape[1]
    287 if X.shape[1] != n_features:
--> 288     raise ValueError("X has %d features per sample; expecting %d"
   

In [2]:
print(f"Accuracy = {accuracy_score(testdf['label'], testdf['preds'])}")

Accuracy = 0.8920377867746289


## Resave the testdf with Predictions

In [3]:
dump(testdf, models_path / "testdf.joblib")

['/home/camo/projects/NF1_SchwannCell_data_analysis/2.evaluate_models/model_data/lr_plate3_cp_fs_data/testdf.joblib']

## Create Dataframe with coefficients for each Genotype

In [ ]:
featdf = testdf.drop(testdf.index)
featdf.drop(["label", "preds"], inplace=True, axis=1)
featdf = featdf.T
featdf = featdf.reset_index()
featdf = featdf.rename(columns={"index": "features"})

In [ ]:
featdf = pd.concat(
    [pd.DataFrame(lr.coef_.T, columns=le.classes_.tolist()), featdf], axis="columns"
)
featdf["abs_HET"] = featdf["HET"].abs()

## Find Confusion Matrix

In [ ]:
cm3 = pd.crosstab(
    testdf["label"], testdf["preds"], rownames=["True"], colnames=["Predicted"]
)
ax = sns.heatmap(
    cm3,
    annot=True,
    cmap="Blues",
    xticklabels=le.classes_.tolist(),
    yticklabels=le.classes_.tolist(),
)
cbar = ax.collections[0].colorbar
cbar.set_label("Number of Single Cells")
plt.title("Performance predicting Genotype")
plt.savefig(f"{fig_out_path}/lr_conf_mat.png")  # Save the Confusion Matrix

In [4]:
print(
    f"The number of incorrectly classified cells is {(cm3.sum() - np.diag(cm3)).sort_values(ascending=False)}"
)

The number of incorrectly classified cells is Predicted
1    219
2    213
0     48
dtype: int64
